In [ ]:
# Install Sastrawi untuk stemming bahasa Indonesia
!pip install Sastrawi

In [ ]:
# Import library
import pandas as pd
import numpy as np
import nltk
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from google.colab import drive
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
# Mount Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Download NLTK stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Load dataset (Excel format)
dataset_path = '/content/drive/MyDrive/NLP/youtube-comments.csv'
df = pd.read_csv(dataset_path)
df.head(20)

,publishedAt,authorDisplayName,textDisplay,likeCount
0,2024-10-30T14:40:13Z,@a.r.silaen3081,"Hai pemilik akun fufufafa, segeralah mundur se...",0
1,2024-10-30T14:29:33Z,@yudhisasminto1800,Pemilik akun fufufafa harus di tindak tegas ag...,0
2,2024-10-30T14:10:21Z,@gojito-kun5062,Susah kali Pronounce fufufafa ya 😂,0
3,2024-10-30T13:59:07Z,@AuroraLesley,Jika Hukum dan aturan konstitusi Negara sdh d...,0
4,2024-10-30T13:53:45Z,@firmanfly8713,Pretttttt,0
5,2024-10-30T13:47:16Z,@bambanghaditjiptono6205,JARKONI bisa ngajar ga bisa ngelakoni. Ngaca c...,0
6,2024-10-30T13:44:14Z,@bambanghaditjiptono6205,Tobat coy dah tua.,0
7,2024-10-30T13:16:34Z,@indrasiregar1179,Sebelum fufufafa d gandeng jd wakil<br>Mereka ...,0
8,2024-10-30T13:14:06Z,@Ernaaprillia-jt1wq,Hukum di per mainkan oleh Jokowi,0
9,2024-10-30T13:11:10Z,@Ernaaprillia-jt1wq,Pak polisi bukan takut sama tuhan TaPI takut s...,0


In [ ]:
df.isna().sum()

,0
publishedAt,0
authorDisplayName,0
textDisplay,0
likeCount,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1678 entries, 0 to 1677
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   publishedAt        1678 non-null   object
 1   authorDisplayName  1678 non-null   object
 2   textDisplay        1678 non-null   object
 3   likeCount          1678 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 52.6+ KB


In [ ]:
df.tail(10)

,publishedAt,authorDisplayName,textDisplay,likeCount
1668,2024-10-29T08:34:43Z,@trititin1278,sekolam jg,1
1669,2024-10-29T08:41:36Z,@PakBambang-g1x,Hanya orang tolol yg takut.... <br>Prabowo men...,0
1670,2024-10-29T08:50:35Z,@azwamumtaz5602,TAKUT LAH SAMA ALLAH . BAUKA SAMA MULJONO .,0
1671,2024-10-29T08:58:25Z,@hormanhadi9699,baru tau kalian yg suka nyinyir sama muliono.d...,0
1672,2024-10-29T07:43:27Z,@AlamSyah-xf1di,Minta maaf dan mundur diri dari wapres. Untuk ...,23
1673,2024-10-29T09:37:24Z,@dewasuardana3310,Berarti anda sdh memastikan pemilik akunnya gi...,0
1674,2024-10-29T14:27:56Z,@Andre_styawan6993,@@AlamSyah-xf1di idiih...wkwkwkwkkw...menghaya...,0
1675,2024-10-29T21:50:06Z,@AlamSyah-xf1di,"Tidak apa-apa, hanya orang yang otaknya kosong...",0
1676,2024-10-29T07:40:42Z,@Sriyatun-lm9lg,Bukan tokoh2 tpi grombolan tikus2 comberan sak...,0
1677,2024-10-29T07:25:09Z,@mbahsakroni4392,Madang,1


In [ ]:
# Data Preprocessing: Pembersihan dataset
def clean_text(text):
  text = text.lower()  # ubah jadi lower_case
  text = re.sub(r"http\S+", "", text)  # hapus URLs
  text = re.sub(r"@\w+", "", text)  # hapus mentions
  text = re.sub(r"#\w+", "", text)  # hapus hashtags
  text = re.sub(r"[^\w\s]", "", text)  # hapus punctuation
  text = re.sub(r"\d+", "", text)  # hilangkan numbers
  text = text.strip()  # Remove leading/trailing whitespaces
  return text

In [ ]:
# Apply dataset yang sudah dibersihkan ke dataset
df['clean_text'] = df['textDisplay'].apply(clean_text)

df[['textDisplay','clean_text']]

,textDisplay,clean_text
0,"Hai pemilik akun fufufafa, segeralah mundur se...",hai pemilik akun fufufafa segeralah mundur seb...
1,Pemilik akun fufufafa harus di tindak tegas ag...,pemilik akun fufufafa harus di tindak tegas ag...
2,Susah kali Pronounce fufufafa ya 😂,susah kali pronounce fufufafa ya
3,Jika Hukum dan aturan konstitusi Negara sdh d...,jika hukum dan aturan konstitusi negara sdh d...
4,Pretttttt,pretttttt
...,...,...
1673,Berarti anda sdh memastikan pemilik akunnya gi...,berarti anda sdh memastikan pemilik akunnya gi...
1674,@@AlamSyah-xf1di idiih...wkwkwkwkkw...menghaya...,xfdi idiihwkwkwkwkkwmenghayal ye wkwkwkwkwk
1675,"Tidak apa-apa, hanya orang yang otaknya kosong...",tidak apaapa hanya orang yang otaknya kosong t...
1676,Bukan tokoh2 tpi grombolan tikus2 comberan sak...,bukan tokoh tpi grombolan tikus comberan sakit...


In [ ]:
# Hapus Stopword
stop_words = set(stopwords.words('indonesian'))
df['clean_text'] = df['clean_text'].apply(lambda x: ' '.join([word
for word in x.split() if word not in stop_words]))

In [ ]:
# Stemming dengan Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
# Apply stemming ke dataset
df['clean_text'] = df['clean_text'].apply(lambda x: ''.join([stemmer.stem(word) for word in x.split()]))

In [ ]:
df[['textDisplay','clean_text']]

,textDisplay,clean_text
0,"Hai pemilik akun fufufafa, segeralah mundur se...",haimilikakunfufufafasegeramundurmakzulkeluarga...
1,Pemilik akun fufufafa harus di tindak tegas ag...,milikakunfufufafatindakduridagingdagingmembusu...
2,Susah kali Pronounce fufufafa ya 😂,susahkalipronouncefufufafaya
3,Jika Hukum dan aturan konstitusi Negara sdh d...,hukumaturkonstitusinegarasdhkacanginkangkangku...
4,Pretttttt,pretttttt
...,...,...
1673,Berarti anda sdh memastikan pemilik akunnya gi...,sdhmilikakungibrantolongbuktiomondoangkayakgon...
1674,@@AlamSyah-xf1di idiih...wkwkwkwkkw...menghaya...,xfdiidiihwkwkwkwkkwmenghayalyewkwkwkwkwk
1675,"Tidak apa-apa, hanya orang yang otaknya kosong...",apaapaorangotakkosonghayalan
1676,Bukan tokoh2 tpi grombolan tikus2 comberan sak...,tokohtpigrombolantikuscomberansakithatiampraya...


# **TF-IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tes = vectorizer.fit_transform(df['clean_text'])
print(tes)
df_final = pd.DataFrame(tes.toarray().T,
                  index=vectorizer.get_feature_names_out(),
                  columns=[f'D{i+1}' for i in range(tes.shape[0])])

  (0, 566)	1.0
  (1, 910)	1.0
  (2, 1414)	1.0
  (3, 616)	1.0
  (4, 1206)	1.0
  (5, 657)	1.0
  (6, 1475)	1.0
  (7, 387)	1.0
  (8, 617)	1.0
  (9, 1147)	1.0
  (10, 780)	1.0
  (11, 247)	1.0
  (12, 78)	1.0
  (13, 630)	1.0
  (14, 1078)	1.0
  (15, 1522)	1.0
  (16, 1062)	1.0
  (17, 465)	1.0
  (18, 1608)	1.0
  (19, 711)	1.0
  (20, 1072)	1.0
  (21, 1333)	1.0
  (22, 1403)	1.0
  (23, 1627)	1.0
  (24, 1150)	1.0
  :	:
  (1652, 144)	1.0
  (1653, 819)	1.0
  (1654, 267)	1.0
  (1655, 958)	1.0
  (1656, 622)	1.0
  (1657, 1446)	1.0
  (1658, 182)	1.0
  (1659, 1506)	1.0
  (1660, 1532)	1.0
  (1661, 639)	1.0
  (1662, 487)	1.0
  (1663, 346)	1.0
  (1664, 849)	1.0
  (1666, 1465)	1.0
  (1667, 1435)	1.0
  (1668, 788)	1.0
  (1669, 1086)	1.0
  (1670, 1433)	1.0
  (1671, 1452)	1.0
  (1672, 834)	1.0
  (1673, 1316)	1.0
  (1674, 1598)	1.0
  (1675, 135)	1.0
  (1676, 1481)	1.0
  (1677, 836)	1.0


In [ ]:
vectorizer.get_feature_names_out()

array(['aaajamnmaklukgoibikutn',
       'aaenggimcaripamoryaaudahlaahurusinpesantren',
       'aageamngapainngurusinuruskayagitusamaajakiyainguyrusinkebencianmendingpolitikajageamcobanyapreslakuygkalaungakukiyaidakwaygbenerkiyaingurusinbenci',
       ...,
       'yiygdiememasjdtakuttdkbelaklkerjayautkspypuruksikapblumdewasanegaraklgibrandgnsikaprendahhatinyamaafpubliccitratdkrusak',
       'yybenarnyaakunfufufafayyusut',
       'zamanteknologimajulakulidimudahungkaptsbtinggaltunggulangkahyaambilptesiden'],
      dtype=object)

In [ ]:
data.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
df_final = pd.DataFrame(data.toarray().T,
                  index=vectorizer.get_feature_names_out(),
                  columns=[f'D{i+1}' for i in range(data.shape[0])])

df_final

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,D1669,D1670,D1671,D1672,D1673,D1674,D1675,D1676,D1677,D1678
aaajamnmaklukgoibikutn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaenggimcaripamoryaaudahlaahurusinpesantren,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aageamngapainngurusinuruskayagitusamaajakiyainguyrusinkebencianmendingpolitikajageamcobanyapreslakuygkalaungakukiyaidakwaygbenerkiyaingurusinbenci,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aagimceramahkampanyepolitik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aagimenggapikirfufupapapikirngomongpolitikorangoranggilabicarafufupapa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ygusahameny,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
yiyalhbrosiculungibranygmataklihtannarkobaygjdi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
yiygdiememasjdtakuttdkbelaklkerjayautkspypuruksikapblumdewasanegaraklgibrandgnsikaprendahhatinyamaafpubliccitratdkrusak,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
yybenarnyaakunfufufafayyusut,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
